# Wav2lip唇形迁移的介绍

PaddleGAN的视频唇形同步模型Wav2lip实现了人物口型与输入语音同步，也就是俗称的「对口型」。已经有许多有趣的模型应用，比如下面的两位“女神”、“男神”的图片配音：

![](https://ai-studio-static-online.cdn.bcebos.com/3c3e7f0acaa0493ebef06bf5df497894ee0c7e4a35b8472690a7bfd93a687d50)

![](https://ai-studio-static-online.cdn.bcebos.com/09e8a094530842bd8e77ad95d63e1f6eedbe240aa4554334bce8a0ddf28541d0)

Wav2lip实现唇形与语音精准同步突破的关键在于，它采用了唇形同步判别器，以强制生成器持续产生准确而逼真的唇部运动。我们也可以将Wav2lip理解为一种使用脸部图片和语音音频作为监督信息的 Conditional GAN。

![](https://ai-studio-static-online.cdn.bcebos.com/0294230ec6294da5af12d9f033cdd7507e93290abca0425997ed72607aac19e3)

此外，该研究通过在鉴别器中，使用多个连续帧而不是单个帧，并使用视觉质量损失（而不仅仅是对比损失）来考虑时间相关性，从而改善了视觉质量。因此，Wav2lip不但能够给输入的图片，而且能够给输入视频对口型。

该wav2lip模型适用于任何人脸、任何语音、任何语言，对任意视频都能达到很高的准确率，可以无缝地与原始视频融合，还可以用于转换动画人脸，并且导入合成语音也是可行的。因为无论使用何种语言的语料进行训练，口型与发音的对应还是一样的。

当然，wav2lip模型对转化的视频还是有一点点要求的，就是视频帧（图片同样）中必须包含至少一个嘴部较完整面部。毕竟没有嘴也是没法对嘴型的~

# 实现唇形迁移

下面我们就通过一个 demo 来展示如何使用 PaddleGAN 来实现“**5行代码配嘴型**”。

## 1.下载、安装 PaddleGAN
如果 github 较慢可是使用 gitee 上的镜像下载PaddleGAN。下载后进入目录进行安装。安装过程主要是安装依赖和设置环境变量以正常运行 PaddleGAN 集成的各种模型训练、预测脚本。

In [ ]:
!git clone https://gitee.com/paddlepaddle/PaddleGAN.git
%cd PaddleGAN/
!pip install -v -e .

Cloning into 'PaddleGAN'...
remote: Enumerating objects: 2995, done.
remote: Counting objects: 100% (2995/2995), done.
remote: Compressing objects: 100% (1759/1759), done.
remote: Total 2995 (delta 1910), reused 1838 (delta 1146), pack-reused 0
Receiving objects: 100% (2995/2995), 153.76 MiB | 21.98 MiB/s, done.
Resolving deltas: 100% (1910/1910), done.
Checking connectivity... done.
/home/aistudio/PaddleGAN
Created temporary directory: /tmp/pip-ephem-wheel-cache-4e_nwou_
Created temporary directory: /tmp/pip-req-tracker-9hfeczq0
Created requirements tracker '/tmp/pip-req-tracker-9hfeczq0'
Created temporary directory: /tmp/pip-install-020vaeh_
Looking in indexes: https://mirror.baidu.com/pypi/simple/
Obtaining file:///home/aistudio/PaddleGAN
  Added file:///home/aistudio/PaddleGAN to build tracker '/tmp/pip-req-tracker-9hfeczq0'
    Running setup.py (path:/home/aistudio/PaddleGAN/setup.py) egg_info for package from file:///home/aistudio/PaddleGAN
    Running command python setup.py egg

## 2.准备素材

* 在项目目录下建立 audio 文件夹用来存储生成嘴型的音频文件。这里使用“第8套广播体操”的伴奏音频，文件名为 gymnastics.m4a 。m4a是win10录音用的默认格式，当然也可以使用 wav 和 mp3 格式的音频文件。

* 在项目目录下建立 face 文件夹用来存储进行对嘴型的视频或图片。这里使用一个《疯狂的外星人》中的片段，文件名为 alien.mp4。

In [5]:
# 这段代码用于展示视频不算在那“5行代码配嘴型”之内~
import cv2
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

def display(driving, fps, size=(8, 6)):
    fig = plt.figure(figsize=size)

    ims = []
    for i in range(len(driving)):
        cols = []
        cols.append(driving[i])

        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    video = animation.ArtistAnimation(fig, ims, interval=1000.0/fps, repeat_delay=1000)

    plt.close()
    return video

input_video_path = '/home/aistudio/face/alien.mp4'

video_frames = imageio.mimread(input_video_path, memtest=False)

# 获得视频的原分辨率
cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
    

HTML(display(video_frames, fps).to_html5_video())

## 3.进行唇形合成

首先进入 PaddleGAN 的 applications 目录，然后调用 tools 文件夹下的 wav2lip.py 脚本进行唇形合成。

--face 参数指定进唇形行合成的图片或视频。

--audio 参数指定口型依据的音频。

--outfile 参数指定输出的合成好的视频文件路径。


In [4]:
%cd /home/aistudio/PaddleGAN/applications/
!python tools/wav2lip.py --face /home/aistudio/face/alien.mp4 --audio /home/aistudio/audio/gymnastics.m4a --outfile /home/aistudio/output/output.mp4


/home/aistudio/PaddleGAN/applications
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/scipy/linalg/__init_

## 4.查看合成的视频

合成的视频存放在 --outfile 参数指定的 /home/aistudio/output 文件夹下。合成的视频已经配好口型，并替换了音轨。如果打开时间过长或想听音频效果，请将合成的文件下载。

这里展示下合成视频配嘴型的效果，看看“地外总代理”是怎么忽悠外星人的吧~

In [6]:
output_video_path = '/home/aistudio/output/output.mp4'

video_frames = imageio.mimread(output_video_path, memtest=False)

# 获得视频的原分辨率
cap = cv2.VideoCapture(output_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
    

HTML(display(video_frames, fps).to_html5_video())